In [1]:
import chromadb
import json
from langchain.embeddings import OpenAIEmbeddings
import openai
openai.api_key='sk-proj-9oFspACwXLo6tjgsobG7T3BlbkFJIXBM10NHcdXgdU5F9nQH'
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-9oFspACwXLo6tjgsobG7T3BlbkFJIXBM10NHcdXgdU5F9nQH'
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings()
embedding_function = embeddings



In [3]:
#### Creating chroma Collection
def create_chroma_index(json_file_path = '.json', collection_name = '', index_key = None ):
    

    f=open(json_file_path)
    data_hadith=json.load(f)
    f.close()
    json_data = data_hadith
    docs = []
    embedding_data = []


    if index_key == None :
        index_key = collection_name


    for index in range(len(data_hadith[index_key])):
        docs.append(data_hadith[index_key][index])
        embedding_data.append(data_hadith["openai_embeddings"][index])
    persistent_client = chromadb.PersistentClient()
    collection = persistent_client.create_collection(collection_name,
                                                    metadata={"hnsw:space": "l2"})
    collection.add( ids = [str(i) for i in range(len(docs))], documents=docs, embeddings=embedding_data)
    print(f"created {collection_name}")
    langchain_chroma = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embedding_function,
    )
    return langchain_chroma

In [2]:
create_chroma_index(json_file_path='Islam_Gordon.json',collection_name = 'Islam_Gordon' )

NameError: name 'create_chroma_index' is not defined

In [15]:
def similarity_search_score(query= "", collection_name = '', vector = None ):
        from langchain.embeddings import OpenAIEmbeddings
        from langchain.vectorstores import Chroma
        import chromadb
        index_key = None
        persistent_client = chromadb.PersistentClient()
        # collection = persistent_client.get_collection(collection_name)
        embeddings =  OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
        langchain_chroma = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        )
        if vector:
            return collection_name, langchain_chroma.similarity_search_by_vector(vector, k = 3)
        return collection_name, langchain_chroma.similarity_search_with_score(query, k = 3)

In [ ]:
def get_context_page(self,question,collection_name):
        val=self.similarity_search_score(question,collection_name[0])
        val2= self.similarity_search_score(question,collection_name[1])
        map_context, map_context2= {}, {}
        for i, value in enumerate(val[1]):
            map_context[value[0].page_content]=value[1]
        for i, value in enumerate(val2[1]):
            map_context2[value[0].page_content]=value[1]

        val=sorted(map_context, key=map_context.get, reverse=True)[:4]
        val2=sorted(map_context2, key=map_context2.get, reverse=True)[:4]

        val_map={}
        val_map[f"{collection_name[0]}_A"] = val[0]
        val_map[f"{collection_name[0]}_B"] = val[1]
        val_map[f"{collection_name[1]}_A"] = val2[0]
        val_map[f"{collection_name[1]}_B"] = val2[1]

        ##Combination of all the values
        context=""
        for con in list(val_map.keys()):
            context= context+ f"{val_map[con]}{con}\n" +"\n"

        return val_map, context

In [16]:
s=similarity_search_score("hi","")

KeyboardInterrupt: 

In [12]:
s[1][0].page_content

AttributeError: 'tuple' object has no attribute 'page_content'